# 调用摄像头实时MobileNet图像分类

>同济大学 张子豪 2021-09-23

视频教程：同济子豪兄 https://space.bilibili.com/1900783

# 导入工具包、预训练模型

In [1]:
# 导入工具包

import numpy as np

# 导入opencv-python
import cv2

# 载入图像处理、预处理、解码预测结果的辅助函数
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions

# 载入预训练模型，执行图像分类

# # 导入MobileNet V1
# from tensorflow.keras.applications import MobileNet
# model = MobileNet(alpha=1.0,                      # MobileNet论文中的宽度超参数
#                 depth_multiplier=1,               # MobileNet论文中的分辨率超参数
#                 include_top=True,                 # 是否包含最后的全连接分类层
#                 weights="imagenet",               # 权重默认为ImageNet一千类图像分类
#                 classes=1000                      # 1000个类别
# )

# 导入MobileNet V2
from tensorflow.keras.applications import MobileNetV2
model = MobileNetV2(alpha=1.0,                      # MobileNet论文中的宽度超参数
                include_top=True,                 # 是否包含最后的全连接分类层
                weights="imagenet",               # 权重默认为ImageNet一千类图像分类
                classes=1000                      # 1000个类别
)

# # 导入ResNet50
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
# model = ResNet50(include_top=True,weights="imagenet",classes=1000)

# 处理单帧图像的函数

In [2]:
def process_frame(img):
    
    '''
    B站：同济子豪兄（https://space.bilibili.com/1900783）

    微信公众号：人工智能小技巧
    '''
    
    # 记录该帧开始处理的时间
    start_time = time.time()
    
    #################################################
    
    img_copy = img
    
    # BGR转RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # 缩放至224x224
    img = cv2.resize(img, (224,224))
    x = np.expand_dims(img, axis=0)
    # 预处理
    x = preprocess_input(x)

    # 获取图像分类预测结果
    preds = model.predict(x)
    results = decode_predictions(preds, top=5)
    
    # 遍历results变量中的每一个元素，写在原图左上角
    for idx, result in enumerate(results[0]): 
        text1 = '{:<10} {:>.4f}'.format(result[1], result[2])

        # !图片，添加的文字，左上角坐标，字体，字体大小，颜色，字体粗细
        img = cv2.putText(img_copy, text1, (25, 80 + 40 * idx), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (255, 255, 255), 2)
    #################################################
            
    # 记录该帧处理完毕的时间
    end_time = time.time()
    # 计算每秒处理图像帧数FPS
    FPS = 1/(end_time - start_time)
    
    scaler = 1
    # 在图像上写FPS数值，参数依次为：图片，添加的文字，左上角坐标，字体，字体大小，颜色，字体粗细
    img = cv2.putText(img_copy, 'FPS  '+str(int(FPS)), (25 * scaler, 50 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
    
    return img_copy

# 调用摄像头获取每帧（模板）

In [3]:
# 调用摄像头逐帧实时处理模板
# 不需修改任何代码，只需定义process_frame函数即可
# 同济子豪兄 2021-7-8

# 导入opencv-python
import cv2
import time

# 获取摄像头，传入0表示获取系统默认摄像头,mac系统应改成1
cap = cv2.VideoCapture(0)

# 打开cap
cap.open(0)

# 无限循环，直到break被触发
while cap.isOpened():
    # 获取画面
    success, frame = cap.read()
    if not success:
        break
    
    ## !!!处理帧函数
    frame = process_frame(frame)
    
    # 展示处理后的三通道图像
    cv2.imshow('my_window', frame)

    if cv2.waitKey(1) in [ord('q'),27]: # 按键盘上的q或esc退出（在英文输入法下）
        break
    
# 关闭摄像头
cap.release()

# 关闭图像窗口
cv2.destroyAllWindows()